In [0]:
%sql
select * from gizmobox.bronze.v_memberships;

In [0]:
%sql
select regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id, content as membership_card from gizmobox.bronze.v_memberships;

In [0]:
%sql
create table gizmobox.silver.memberships    --CTAS statement
as
select regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id, content as membership_card from gizmobox.bronze.v_memberships;

In [0]:
%sql
select * from gizmobox.silver.memberships;

### Transform the Addresses Data

Here a customer has both the shipping and billing address in two separate rows, we're going to use PIVOT to make them into one single row

In [0]:
%sql
select * from gizmobox.bronze.v_addresses;

In [0]:
%sql
select * from (
  select customer_id, address_type, address_line_1, city, state, postcode from gizmobox.bronze.v_addresses
)
pivot (max(address_line_1) AS address_line_1,
max (city) as city,
max(state) as state,
max(postcode) as postcode
for address_type IN ('shipping', 'billing'));

In [0]:
%sql
create table gizmobox.silver.addresses
as
select * from (
  select customer_id, address_type, address_line_1, city, state, postcode from gizmobox.bronze.v_addresses
)
pivot (max(address_line_1) AS address_line_1,
max (city) as city,
max(state) as state,
max(postcode) as postcode
for address_type IN ('shipping', 'billing'));

In [0]:
%sql
select * from gizmobox.silver.addresses;

### Query Orders Data as JSON Strings

In [0]:
%sql
select * from gizmobox.bronze.v_orders;

In [0]:
%sql
--- Extracting top level object values

select value:order_id as order_id, value from gizmobox.bronze.v_orders;  ---shows null for invaid json values

In [0]:
%sql
---- Extract Array Value

select value:items[0] as items_1, value:items[1] as items_2, value from gizmobox.bronze.v_orders;

In [0]:
%sql
--- Extract Nested Column values
select value:items[0].item_id as item_1_item_id, value:items[0] as item_1, value:items[1] as item_2, value from gizmobox.bronze.v_orders;

In [0]:
%sql
---CAST column values to a specific data type
select value:items[0].item_id :: INTEGER as item_1_item_id, value:items[0] as item_1, value:items[1] as item_2, value from gizmobox.bronze.v_orders;

As you've seen above the process is timestaking and not recommended, hence we'll convert the above data to JSON object and do it.

### Transform Orders Data- String to JSON

In [0]:
%sql

--- 1. Pre Process the data to fix data quality issues
create or replace temporary view tv_orders_fixed
as
select value, regexp_replace(value, '"order_date": (\\d{4}-\\d{2}-\\d{2})', '"order_date": "\$1"') as fixed_value from gizmobox.bronze.v_orders;

### Transforming JSON String to JSON object
* Function of schema of json
* Function from json

In [0]:
%sql
select schema_of_json(fixed_value) AS schema, --Gets the schema of the json string
fixed_value from tv_orders_fixed limit 1;

In [0]:
%sql
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value,
fixed_value
from tv_orders_fixed;

In [0]:
%sql

---- Transforming the above result to silver schema table

create table if not exists gizmobox.silver.orders_json
as
select from_json(fixed_value, 'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as json_value
from tv_orders_fixed;

In [0]:
%sql
select * from gizmobox.silver.orders_json;

### Transform Order Data - Explode Arrays

In [0]:
%sql
select 
json_value.customer_id,
json_value.order_id,
json_value.order_date,
json_value.order_status,
json_value.payment_method,
json_value.total_amount,
json_value.transaction_timestamp,
json_value.items
from gizmobox.silver.orders_json;

### Deduplicating the arrays in json objects

In [0]:
%sql
select 
json_value.customer_id,
json_value.order_id,
json_value.order_date,
json_value.order_status,
json_value.payment_method,
json_value.total_amount,
json_value.transaction_timestamp,
array_distinct(json_value.items) as items
from gizmobox.silver.orders_json;

### Exploding the arrays to get the values in the nested arrays

In [0]:
%sql
---Syntax : explode(collection)
create or replace temp view tv_orders_explode as
select 
json_value.customer_id,
json_value.order_id,
json_value.order_date,
json_value.order_status,
json_value.payment_method,
json_value.total_amount,
json_value.transaction_timestamp,
explode(array_distinct(json_value.items)) as item
from gizmobox.silver.orders_json;

In [0]:
%sql
create table if not exists gizmobox.silver.orders
as 
select order_id, order_status, payment_method, total_amount, transaction_timestamp, customer_id, item.item_id, item.name, item.quantity, item.category, item.details.brand, item.details.color from tv_orders_explode;

In [0]:
%sql
select * from gizmobox.silver.orders;